Setting up modules

In [3]:
import ee
import geemap
import geopandas as gpd
import requests
from shapely import geometry

ee.Authenticate()
# ee.Initialize()
ee.Initialize(project="ee-fasoriano")



Successfully saved authorization token.


Choosing the area of interest (AOI)

In [12]:
# URL to the GeoJSON file on GitHub
geojson_url = 'https://raw.githubusercontent.com/fralasor/autocam-workshop/main/rsws_region_bboxes.geojson'

# Fetch the GeoJSON file
geojson_data = requests.get(geojson_url).json()

# Create a map instance
map1 = geemap.Map()

# Load GeoJSON into a GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])
# display(gdf)

# Select the AOI based on its ID
feature = gdf[gdf['id'] == 0].squeeze()

# Convert the selected feature geometry to a Shapely object
geom = feature.geometry

# Convert Shapely geometry to GeoJSON
geom_geojson = geometry.mapping(geom)
aoi_bbox = ee.FeatureCollection(ee.Geometry(geom_geojson))

# AOI Visualization
aoi_style = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# Add the selected AOI to the map
map1.addLayer(aoi_bbox.style(**aoi_style), {}, "Selected AOI")
map1.centerObject(aoi_bbox)

# Display the map
map1


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Parameters

In [13]:
"""
You can also use the two following lines to set your AOI based on a shp/json file  
# extent_path = fr"D:/fsori/Documents/AUTOCAM/Study Area/tagum_boundbox.geojson"
# AOI = geemap.geojson_to_ee(extent_path)
"""

AOI = aoi_bbox
START_DATE = '2023-01-01'
END_DATE = '2023-03-31'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST_km = 2
BUFFER_m = 100
output_name = 'tagum_2023_q1_s2'

Cloud masking using S2 cloud probability dataset (s2cloudless) dataset

In [14]:
# Defining functions for cloud-masking
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))
    
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST_km input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST_km*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER_m*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

# Sentinel 2 Visualization
s2_visualization = {
    'min': 0,
    'max': 10000,
    'bands': ['B4', 'B3', 'B2'],
    'gamma': 1.2
}


Calling S2 image collection & Adding cloud-free composites to basemap

In [15]:
# Fetching Sentinel 2 image collection within the set time period
s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(AOI)
        .filterDate(START_DATE, END_DATE)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
        )

# APPLYING S2 CLOUD MASK
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

# Add median cloud-free composite
map1.addLayer(s2_sr_median.clip(aoi_bbox), s2_visualization, name='s2 median cloudless')

# Bring AOI polygon to top
map1.remove(aoi_bbox)
map1.addLayer(aoi_bbox.style(**aoi_style), {}, "Selected AOI")

# Display map
map1

Map(center=[10.505025167387853, 122.99509115650005], controls=(WidgetControl(options=['position', 'transparent…

In [16]:
print('All band names:', s2_sr_median.bandNames().getInfo())

All band names: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']


Loading training and testing data

In [ ]:
train_data_path = 
test_data_path = 

# Fetch the GeoJSON file
train_data = requests.get(train_data_path).json()
test_data = requests.get(test_data_path).json()

print(f"train data is {type(train_data)}")

lc_legend_dict = {
    'inland water', 
    'grassland', 
    'fallow land', 
    'crops', 
    'coastal water', 
    'built-up', 
    'barren', 
    'trees', 
    'brush', 
    'fishpond', 
    'mangrove'
    }

In [ ]:
bands = s2_sr_median.bandNames().getInfo()
label = "lc_code"

training = s2_sr_median.select(bands).sampleRegions(**{
    'collection':train_data,
    'properties':[label],
    'scale':10,
    # 'tileScale': 16 # TRY LANG!!!
})

classifier = ee.Classifier.smileRandomForest(100).train(training, label, bands) # .train()

result = s2_sr_median.select(bands).classify(classifier) #.classify() 'trained' => trained classifier

# Map.addLayer(study_area, opacity=0.4)
map.addLayer(result.clip(AOI), rf_vis_param, 'classified')

In [ ]:
#Validation

validation = s2_sr_median.select(bands).sampleRegions(**{
    'collection':test_data,
    'properties':[label],
    'scale':10,
})

test = validation.classify(classifier)

confusionMatrix = test.errorMatrix('lc_code','classification')

print('RF Consumer\'s accuracy (Omission)', confusionMatrix.consumersAccuracy().getInfo())
print('RF Producer\'s accuracy (Commission)', confusionMatrix.producersAccuracy().getInfo())
print('RF Overall accuracy', confusionMatrix.accuracy().getInfo())
print('RF Kappa Coefficient', confusionMatrix.kappa().getInfo())
print('RF testing error matrix', confusionMatrix.getInfo())


Exporting satellite image to Google Drive

In [ ]:
task = ee.batch.Export.image.toDrive(**{
    'image': s2_sr_median.clip(aoi_bbox),
    'description': output_name,
    'folder': 'autocam_s2',
    'scale': 10,
    'region': aoi_bbox.geometry(),
    'crs': 'EPSG:4326'
    
})

task.start()
print(f"Running task {task.status()['task_type']} of {task.status()['description']}")

while(task.status()['state'] != 'COMPLETED'):
    continue

task.status()
